<a href="https://colab.research.google.com/github/Ricardomanuel1/Maestria_Ciencia_de_Datos/blob/main/II.%20Introduccion_Ciencia_Datos/EjercicioAdvertising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Regresión  - Ejercicio Advertising

El objetivo es usar regresión probablemente multivariada para entender como los gastos en anuncios impactan las ventas (variable a predecir es **sales**).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
url = "https://raw.githubusercontent.com/eduardool/MaestriaCienciaDatos/main/datasets/advertising_regresion.csv"

In [3]:
datos = pd.read_csv(url)

In [4]:
datos.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [5]:
datos=datos.drop("Unnamed: 0",axis=1)

In [6]:
datos.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [7]:
datos.shape

(200, 4)

In [8]:
datos.info()#no se tienen datos nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   radio      200 non-null    float64
 2   newspaper  200 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [9]:
datos.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,200.0,147.0425,85.854236,0.7,74.375,149.75,218.825,296.4
radio,200.0,23.2640,14.846809,0.0,9.975,22.90,36.525,49.6
newspaper,200.0,30.5540,21.778621,0.3,12.750,25.75,45.100,114.0
sales,200.0,14.0225,5.217457,1.6,10.375,12.90,17.400,27.0


In [10]:
datos.columns

Index(['TV', 'radio', 'newspaper', 'sales'], dtype='object')

**Preparamos los datos para los algoritmos de ML**

In [11]:
X=datos[['TV','radio','newspaper']]
y=datos['sales']

Separación de los datos en conjunto de entrenamiento y prueba

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=100)

In [14]:
X_entrenamiento.shape

(160, 3)

In [15]:
X_prueba.shape

(40, 3)

Estandarización del conjunto de entrenamiento

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
X_preparado=StandardScaler().fit_transform(X_entrenamiento)

In [18]:
#X_preparado

#MODELO DE REGRESION LINEAL

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
reg_lin = LinearRegression()

reg_lin.fit(X_preparado, y_entrenamiento)

LinearRegression()

In [21]:
# prueba con algunas instancias pequeñas
pocos_X = X_entrenamiento.iloc[:5]
pocos_y = y_entrenamiento.iloc[:5]

pocos_datos_preparados = StandardScaler().fit_transform(pocos_X)

print("Predicciones:", reg_lin.predict(pocos_datos_preparados))

Predicciones: [20.90463152 12.69013752 13.89650424 13.31565588  8.18682084]


In [22]:
print("y real:", list(pocos_y))

y real: [17.1, 14.1, 14.5, 12.2, 13.2]


In [23]:
predicciones_X = reg_lin.predict(X_preparado)

In [24]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(y_entrenamiento, predicciones_X)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1.7518885936753472

In [25]:
from sklearn import metrics

In [26]:
metrics.mean_squared_error(y_entrenamiento, predicciones_X)

3.0691136446497858

In [27]:
print('MAE:', metrics.mean_absolute_error(y_entrenamiento, predicciones_X))
print('MSE:', metrics.mean_squared_error(y_entrenamiento, predicciones_X))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_entrenamiento, predicciones_X)))

MAE: 1.3491729189156043
MSE: 3.0691136446497858
RMSE: 1.7518885936753472


#MODELO DE RANDOM FOREST

In [28]:
from sklearn.ensemble import RandomForestRegressor

reg_rf = RandomForestRegressor(n_estimators=100, random_state=42)
reg_rf.fit(X_preparado, y_entrenamiento)

RandomForestRegressor(random_state=42)

In [29]:
predicciones_X_forest = reg_rf.predict(X_preparado)

In [30]:
print('MAE:', metrics.mean_absolute_error(y_entrenamiento, predicciones_X_forest))
print('MSE:', metrics.mean_squared_error(y_entrenamiento, predicciones_X_forest))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_entrenamiento, predicciones_X_forest)))

MAE: 0.22690625000000067
MSE: 0.10070078124999982
RMSE: 0.3173338640139117


#EVALUACIÓN CRUZADA

Evaluación cruzada el modelo de regresión lineal multivariada

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
lin_scores = cross_val_score(reg_lin, X_preparado, y_entrenamiento,
                             scoring="neg_mean_squared_error", cv=5)
lin_rmse_scores = np.sqrt(-lin_scores)

In [39]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [40]:
display_scores(lin_rmse_scores)

Scores: [1.39824197 1.80189657 2.05510236 1.70522543 2.21153457]
Mean: 1.834400181974481
Standard deviation: 0.28245933644475923


Evaluación cruzada para el modelo de random forest

In [44]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(reg_rf, X_preparado, y_entrenamiento,
                                scoring="neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-forest_scores)

display_scores(forest_rmse_scores)

Scores: [0.78134789 0.81435271 0.79375976 0.81229145 1.32085486]
Mean: 0.904521337334568
Standard deviation: 0.2085234556506384


#AFINAMIENTO DEL MODELO CON MEJOR DESEMPEÑO RANDOM FOREST

In [54]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # Se prueba 12 (3×4) combinaciones de hiperparámetros
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # entonces intentar 6 (2×3) combinaciones con bootstrap en False (sampling con reemplazo y sin reemplazo)
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

reg_forest = RandomForestRegressor(random_state=100)
# Al usar cv  5 en total son (12+6)*5=90 entrenamientos
grid_search = GridSearchCV(reg_forest, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_preparado, y_entrenamiento)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=100),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [60]:
grid_search.best_params_

{'max_features': 4, 'n_estimators': 30}

In [61]:
grid_search.best_estimator_

RandomForestRegressor(max_features=4, n_estimators=30, random_state=100)

Mejor modelo

In [62]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1.3778556205608452 {'max_features': 2, 'n_estimators': 3}
1.166394551599072 {'max_features': 2, 'n_estimators': 10}
1.0298044582454584 {'max_features': 2, 'n_estimators': 30}
1.1067877694782622 {'max_features': 4, 'n_estimators': 3}
1.007810436044398 {'max_features': 4, 'n_estimators': 10}
0.9434643975500319 {'max_features': 4, 'n_estimators': 30}
1.1067877694782622 {'max_features': 6, 'n_estimators': 3}
1.007810436044398 {'max_features': 6, 'n_estimators': 10}
0.9434643975500319 {'max_features': 6, 'n_estimators': 30}
1.1067877694782622 {'max_features': 8, 'n_estimators': 3}
1.007810436044398 {'max_features': 8, 'n_estimators': 10}
0.9434643975500319 {'max_features': 8, 'n_estimators': 30}
1.6368754109922694 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
1.2684153499544224 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
1.3240090634130874 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
1.3152730229880032 {'bootstrap': False, 'max_features': 3, 'n_est

Busqueda aleatoria de hiperparametros

In [64]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

reg_forest = RandomForestRegressor(random_state=100)
rnd_search = RandomizedSearchCV(reg_forest, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=100)
rnd_search.fit(X_preparado, y_entrenamiento)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=100),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c6f84ff1a80>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7c6f84323160>},
                   random_state=100, scoring='neg_mean_squared_error')

Seleccion del modelo final

In [81]:
modelo_final = grid_search.best_estimator_

In [80]:
X_prueba_preparado = StandardScaler().fit_transform(X_prueba)
predicciones_finales = modelo_final.predict(X_prueba_preparado)

final_mse = mean_squared_error(y_prueba, predicciones_finales)
final_rmse = np.sqrt(final_mse)
final_rmse

1.9859486258321093

Separar $X$ y $y$

Separar conjunto de entrenamiento (80\%) y prueba (20\%), random_state = 100

Calcular y realizar la escala de variables sobre el conjunto entrenamiento.

Usar validación cruzada (5-folds) para comparar los algoritmos de regresión lineal multivariada y Random forest. En ambos algoritmos usar un random_state = 100. Cual algoritmo es mejor?

Realizar el afinamiento del algoritmo que tuvo mejor desempeño (usar GridSearchCV).


Reportar el RMSE del algoritmo afinado sobre el conjunto de prueba (no olvidarse de antes aplicar la escala calculada sobre el conjunto de entrenamiento).